# Tracing Clues - The Orca Project

The goal of this project is to get the data from the [Orca Network](http://www.orcanetwork.org/Main/) and create a data model that would allow data insights along with building a knowledge graph. Please note this was mostly a scratch pad to learn how to do the different steps. Feel free to play around with it though to get an idea what I was up to. It is rarther messy towards the end though.

There are several stages that will need to be done, but to start we need to grab the data. This website hard codes the sightings into nested tables in html. Right now the plan is to use the Beautiful Soup Library to scrape it. 

Check out the design and what I'm up to here: https://www.craft.do/s/Ly2CiCmBi6h4Mv

## Getting the Website

Note that the year 2001 has a different format and it will need to have a bit of a different script. As will the current sightings page. Although we could do somehting like only bring in the new sightings after a month has completed. 

In [72]:
import requests

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
page = requests.get("http://www.orcanetwork.org/Archives/index.php?categories_file=Sightings%20Archive%20-%20Oct%2005", headers=headers)
page


<Response [200]>

## Beautiful Soup

In [73]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')


In [74]:
soup

<!DOCTYPE doctype html public "-//w3c//dtd html 3.2//en">

<html>
<head>
<title>
Welcome to Orca Network - Sightings Archive - Oct 05</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Orca Network: Enhancing awareness of the Southern Resident Orca (killer whale) community to foster a stewardship ethic to protect and restore orca habitat." name="description"/>
<meta content="Orcinus orca, orca, killer whale, Puget Sound, Salish Sea, salmon, Orca Network, Southern Resident community, J pod, K pod, L pod, orca gifts, orca, donate, gray whale, Baja, San Ignacio, orca protection, orca habitat, whale research, San Juan Island, Whidbey Island" name="keywords"/>
<meta content="Howard Garrett" name="author"/>
<meta content="Welcome to Orca Network" property="og:title">
<meta content="http://www.orcanetwork.org/Archives" property="og:url">
<meta content="http://www.orcanetwork.org/Archives/Images/FB.jpg" property="og:image">
<meta content="Welcome to Orc

In [75]:
for br_tag in soup.find_all('!--'):
    print(br_tag.text, br_tag.next_sibling)

In [76]:
soup.find_all('h3')[0].getText()

'October 2005 Whale Sightings'

Here's where I finally found a way to get all of the data. 

In [77]:
divTag = soup.find("div")
children = divTag.findChildren()
for child in children:
    print(child)

<a name="top"></a>
<h3><center>October 2005 Whale Sightings</center></h3>
<center>October 2005 Whale Sightings</center>
<br/>
<b>October 31, 2005</b>
<br/>
<br/>
<b>orcas</b>
<b>3 pm</b>
<br/>
<br/>
<br/>
<br/>
<b>J pod</b>
<b>1:30 pm</b>
<br/>
<br/>
<b>October 30, 2005</b>
<br/>
<br/>
<b>J pod</b>
<b>3:15 pm</b>
<br/>
<br/>
<b>orcas</b>
<b>11:15 am</b>
<br/>
<br/>
<b>Transients</b>
<b>1030</b>
<br/>
<br/>
<br/>
<br/>
<br/>
<b>orcas</b>
<b>8:40 am</b>
<br/>
<br/>
<b>orcas</b>
<b>8 am</b>
<br/>
<br/>
<b>October 29, 2005</b>
<br/>
<br/>
<b>orcas</b>
<b>5:15 pm</b>
<br/>
<br/>
<b>orcas</b>
<b>4 pm</b>
<br/>
<br/>
<b>orcas</b>
<b>11 am</b>
<br/>
<br/>
<b>orcas</b>
<b>10:20 am</b>
<br/>
<br/>
<br/>
<br/>
<b>October 26, 2005</b>
<br/>
<br/>
<b>orcas</b>
<b>J &amp; L pods</b>
<br/>
<br/>
<b>Orcas</b>
<b>5 PM</b>
<br/>
<br/>
<br/>
<br/>
<b>orcas</b>
<b>5 pm</b>
<br/>
<br/>
<b>J pod</b>
<b>4:20 pm</b>
<br/>
<br/>
<br/>
<br/>
<b>orcas</b>
<b>2:40 pm</b>
<br/>
<br/>
<b>orcas</b>
<b>2:30 pm</b>
<b

Now that we have the data, we need to clean it up. In this case we need to get rid off all the tags so it is only the text left.

In [199]:
import re
pattern = r"<br>||</br>"

cleanUpDiv = re.sub(pattern, "", divTag.getText())
textOnly = re.sub('\n', ' ', cleanUpDiv)
textOnly = re.sub("'", "\'", textOnly)
textOnly = re.sub('"', '\\"', textOnly)
textOnly = re.sub('Clip Map to enlarge    Map © 2005 used with permission byAdvanced Satellite Productions, Inc.', '', textOnly)
textOnly = textOnly.strip()
textOnly

'October 2005 Whale Sightings  October 31, 2005  I went to S. Whidbey after hearing the orcas were headed NW, & found the orcas at 3 pm after they passed Apple Cove Pt. heading toward Point No Point, close to the Kitsap Peninsula side. They were very spread out & far away, & if it weren\'t for one huge male fin (J1\'s I think) I wouldn\'t have spotted them. The visibility was deteriorating, & I couldn\'t find them again as I traveled further north up the island. Susan Orca Network * Tom McMillen of Salish Sea Charters called to report J pod near Pt. Wells (Edmonds area) at 1:30 pm, heading north.  October 30, 2005  Brad Hanson reported J pod very spread out - they left them near Point no Point at 3:15 pm, heading north. * Luann Fortin called to report orcas going from Double Bluff (w. Whidbey) across to Point No Pt (N. Kitsap Peninsula) at 11:15 am. * At least 4 Transients off Victoria at 1030: T103, T104 and T46B with T46B1 Reported by Mark \\"Mallard\\" Malleson. Ron Bates MMRG, Vict

In [163]:
from datetime import date
from dateutil import parser

sample_date = parser.parse('October 27, 2005').date()
month = sample_date.strftime("%B")
year = sample_date.strftime("%Y")
print(year)

searchPattern = re.compile(month + " \d{1,2}, " + year)
splitList = re.split(searchPattern, textOnly)
splitList = splitList[1:]


mapFromDateToReport = dict()
for date, text in zip(re.findall(searchPattern, textOnly), splitList):
    dateObject = parser.parse(date).date()
    mapFromDateToReport[dateObject] = [i.strip() for i in text.split('*')]
    
mapFromDateToReport

2005


{datetime.date(2005, 10, 31): ["I went to S. Whidbey after hearing the orcas were headed NW, & found the orcas at 3 pm after they passed Apple Cove Pt. heading toward Point No Point, close to the Kitsap Peninsula side. They were very spread out & far away, & if it weren't for one huge male fin (J1's I think) I wouldn't have spotted them. The visibility was deteriorating, & I couldn't find them again as I traveled further north up the island. Susan Orca Network",
  'Tom McMillen of Salish Sea Charters called to report J pod near Pt. Wells (Edmonds area) at 1:30 pm, heading north.'],
 datetime.date(2005, 10, 30): ['Brad Hanson reported J pod very spread out - they left them near Point no Point at 3:15 pm, heading north.',
  'Luann Fortin called to report orcas going from Double Bluff (w. Whidbey) across to Point No Pt (N. Kitsap Peninsula) at 11:15 am.',
  'At least 4 Transients off Victoria at 1030: T103, T104 and T46B with T46B1 Reported by Mark "Mallard" Malleson. Ron Bates MMRG, Vict

In [115]:
import datefinder

matches = datefinder.find_dates(textOnly)

for match in matches:
    print(match)




2005-10-18 00:00:00
2005-10-31 00:00:00
2021-11-18 15:00:00
2021-11-18 13:30:00
2005-10-30 00:00:00
2021-11-18 15:15:00
2021-11-18 11:15:00
2021-11-04 00:00:00
1030-11-18 00:00:00
2021-03-18 00:00:00
2021-11-03 00:00:00
2021-11-18 08:40:00
2021-11-18 08:00:00
2021-11-08 00:00:00
2021-11-18 00:00:00
2005-10-29 00:00:00
2021-11-18 17:15:00
2021-11-06 00:00:00
2021-11-18 16:00:00
2021-11-18 11:00:00
2021-06-08 00:00:00
2021-01-18 10:20:00
2005-10-26 00:00:00
2021-11-24 00:00:00
2021-11-18 17:00:00
2021-11-18 17:00:00
2021-11-18 16:20:00
2021-11-18 14:40:00
2021-11-04 00:00:00
2021-11-18 14:30:00
2021-11-18 09:50:00
2021-11-18 08:45:00
2021-11-10 00:00:00
2005-10-25 00:00:00
2021-11-06 00:00:00
2021-11-02 00:00:00
2021-11-18 16:45:00
2021-11-18 15:30:00
2021-11-10 00:00:00
2021-11-05 00:00:00
2021-11-06 00:00:00
2005-10-24 00:00:00
2021-11-06 00:00:00
2021-11-18 17:25:00
2021-11-18 15:00:00
2005-10-23 00:00:00
2021-11-18 17:20:00
2021-11-03 00:00:00
2021-11-18 12:35:00
2021-11-18 11:45:00


In [ ]:
splitList = re.split(searchPattern, textOnly)
splitList = splitList[1:]
print(re.findall(searchPattern, textOnly))
mapFromDateToReport = dict()
for date, text in zip(re.findall(searchPattern, textOnly), splitList):
    mapFromDateToReport[date] = [i.strip() for i in text.split('*')]
    
mapFromDateToReport

Huzzah! Now this can be translated into entries for neo4j.

## Converting to Using Datetime

In [82]:
from datetime import date
from dateutil import parser

sample_date = parser.parse('December 27, 2005').date()
sample_date

datetime.date(2005, 12, 27)

In [33]:
searchPattern = r"December \d{1,2}, \d{4}"
splitList = re.split(searchPattern, textOnly)
splitList = splitList[1:]
mapFromDateToReport = dict()
for date, text in zip(re.findall(searchPattern, textOnly), splitList):
    dateObject = parser.parse(date).date()
    mapFromDateToReport[dateObject] = [i.strip() for i in text.split('*')]
    
mapFromDateToReport

{datetime.date(2005, 12, 27): ['A new Humpback whale was spotted South of Race Rocks this afternoon about 1500. New in the fact its photo id matches nothing taken at least this year. Ron Bates MMRG, Victoria',
  'I am a driver with Prince of Whales. We had a single humpback 3 miles south of Race Rocks. No Orca. Beemer',
  'Marty Tilley called to report many, many orcas off N. Pender Island, traveling south in Swanson Channel at 8:45 am.'],
 datetime.date(2005, 12, 24): ['Mark from the Victoria Clipper called to report a pod of orcas off Pt. Wells near Edmonds, mid-channel heading south at 8:30 am.'],
 datetime.date(2005, 12, 23): ['Tom McMillen called at noon to report a pod of orcas resting just off Shilshole, heading south. At around 1 pm he called to confirm it was J pod, they were at West Pt. heading south slowly. At 1:30 they were off Magnolia, heading south into Elliott Bay.',
  '12:00 PM, On our Seatlle whale watch trip today the Island Explorer II sighted and is presently with 

In [83]:
d = sample_date.strftime("%d %b, %Y")
print("Output 1:", d)

d = sample_date.strftime("%d %B, %Y")
print("Output 2:", d)

d = sample_date.strftime("%B %d, %Y")
print("Output 3:", d)

Output 1: 27 Dec, 2005
Output 2: 27 December, 2005
Output 3: December 27, 2005


## Iterating through the page

In [70]:
start_date = parser.parse("October 1, 2005").date()
end_date = parser.parse("January 1, 2006").date()

In [47]:
from datetime import date

if start_date.year < 2001 or end_date > date.today() or start_date > end_date:
    raise ValueError

In [48]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).months)):
        yield start_date + timedelta(n)

In [50]:
for month in daterange(start_date, end_date):
    print(month)

AttributeError: 'datetime.timedelta' object has no attribute 'months'

In [71]:
import pandas as pd

timedelta_index = pd.date_range(start=start_date, end=end_date, freq='MS', closed=None).to_series()
for index, value in timedelta_index.iteritems():
    dt = index.to_pydatetime().date()
    print(dt)

2005-10-01
2005-11-01
2005-12-01
2006-01-01


## Getting It Into Neo4J

In [122]:
uri = 'neo4j+s://b701f78d.databases.neo4j.io'
user = 'neo4j'
password = '8azuK9q3g8FVR2i96WUts22714vJ76glGMjfNcWCx0g'

In [129]:
from neo4j import GraphDatabase

driver =  GraphDatabase.driver(uri, auth=(user, password))
driver

In [131]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


conn = Neo4jConnection(uri=uri, 
                       user=user,              
                       pwd=password)

In [134]:
conn.query("CREATE (:YEAR {year:2021})")

[]

It worked!!!

In [164]:
mapFromDateToReport
    

{datetime.date(2005, 10, 31): ["I went to S. Whidbey after hearing the orcas were headed NW, & found the orcas at 3 pm after they passed Apple Cove Pt. heading toward Point No Point, close to the Kitsap Peninsula side. They were very spread out & far away, & if it weren't for one huge male fin (J1's I think) I wouldn't have spotted them. The visibility was deteriorating, & I couldn't find them again as I traveled further north up the island. Susan Orca Network",
  'Tom McMillen of Salish Sea Charters called to report J pod near Pt. Wells (Edmonds area) at 1:30 pm, heading north.'],
 datetime.date(2005, 10, 30): ['Brad Hanson reported J pod very spread out - they left them near Point no Point at 3:15 pm, heading north.',
  'Luann Fortin called to report orcas going from Double Bluff (w. Whidbey) across to Point No Pt (N. Kitsap Peninsula) at 11:15 am.',
  'At least 4 Transients off Victoria at 1030: T103, T104 and T46B with T46B1 Reported by Mark "Mallard" Malleson. Ron Bates MMRG, Vict

In [190]:
for curr_date in mapFromDateToReport.keys():
    year = curr_date.strftime("%Y")
    month = curr_date.strftime("%B")
    month_number = int(curr_date.strftime("%m"))
    year_month = curr_date.strftime("%B %Y")
    day = int(curr_date.strftime("%d"))
    day_name = curr_date.strftime("%A")
    full_date = curr_date.strftime("%B %d, %Y")
    
    query_test = '''
                    MERGE (y:Year {date: '%s'})
                    MERGE (m:Month {date: '%s', month: '%s', number: %d})
                    MERGE (d:Day {date: '%s', weekday: '%s', number: %d})
                    CREATE (d)-[:DAY_IN]->(m)-[:MONTH_IN]->(y)
                 '''%(year, year_month, month, month_number, full_date, day_name, day)
    conn.query(query_test)
    break

In [193]:
for date, report_list in mapFromDateToReport.items():
    print(report_list)

["I went to S. Whidbey after hearing the orcas were headed NW, & found the orcas at 3 pm after they passed Apple Cove Pt. heading toward Point No Point, close to the Kitsap Peninsula side. They were very spread out & far away, & if it weren't for one huge male fin (J1's I think) I wouldn't have spotted them. The visibility was deteriorating, & I couldn't find them again as I traveled further north up the island. Susan Orca Network", 'Tom McMillen of Salish Sea Charters called to report J pod near Pt. Wells (Edmonds area) at 1:30 pm, heading north.']
['Brad Hanson reported J pod very spread out - they left them near Point no Point at 3:15 pm, heading north.', 'Luann Fortin called to report orcas going from Double Bluff (w. Whidbey) across to Point No Pt (N. Kitsap Peninsula) at 11:15 am.', 'At least 4 Transients off Victoria at 1030: T103, T104 and T46B with T46B1 Reported by Mark "Mallard" Malleson. Ron Bates MMRG, Victoria', 'Bonnie McKee of Bush Point called to relay a report of at l

The actual queries I did directly in the script, but will include them below. Please note running from 2002 to 2021 took about 1 hour to get into the graph. 

In [200]:
def add_report(connection, date, report):
    full_date = date.strftime("%B %d, %Y")
    query = '''
                MATCH (d:Day {date: '%s'})
                MERGE (s:Sighting {full_text: "%s", processed: 'no'})
                MERGE (s)-[:ON]->(d)
            ''' % (full_date, report)
    connection.query(query)

In [ ]:
def add_time_line(connection, curr_date):
    year = curr_date.strftime("%Y")
    month = curr_date.strftime("%B")
    month_number = int(curr_date.strftime("%m"))
    year_month = curr_date.strftime("%B %Y")
    day = int(curr_date.strftime("%d"))
    day_name = curr_date.strftime("%A")
    full_date = curr_date.strftime("%B %d, %Y")

    query = '''
                MERGE (y:Year {date: '%s'})
                MERGE (m:Month {date: '%s', month: '%s', number: %d})
                MERGE (d:Day {date: '%s', weekday: '%s', number: %d})
                MERGE (d)-[:DAY_IN]->(m)
                MERGE (m)-[:MONTH_IN]->(y)
             ''' % (year, year_month, month, month_number, full_date, day_name, day)
    connection.query(query)

# The Results

The data took about 1 hour to complete for 2002 to 2021. I still need to do 2001 but it has a different format than the rest. 

- Number of Sightings: 33538
- Number of Days with a Report: 3888


Now that means I have 33,538 reports to process by hand to build out the rest of the graph. *facepalm* That's going to take a bit. 